In [227]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pydicom as dicom
from medpy import filter
%matplotlib inline

In [228]:
def display(img,cmap='gray'):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')

In [229]:
#READ THE IMAGE
folder_path = 'DatasetsII/'
filename = '2.jpeg'
ct_liver = cv2.imread(folder_path+'/in/'+filename) 
img = ct_liver.copy()  # DISPLAY
img2 = ct_liver.copy()

In [230]:
#display(ct_liver)

In [231]:
# APPLY BLUR
img = cv2.medianBlur(img,3) # DISPLAY

In [232]:
# display(img)

In [233]:
# CONVERT TO GRAY SCALE
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [234]:
# APPLY THRESHOLD NOT BINARY ==> APPLY OTSU'S METHOD
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)  # DISPLAY

In [235]:
# display(thresh)

In [236]:
# NOISE REMOVAL (OPTIONAL)
kernel = np.ones((3,3),np.uint8)

In [237]:
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel,iterations = 2)     # DISPLAY

In [238]:
# display(opening)

In [239]:
sure_bg = cv2.dilate(opening,kernel,iterations=3)  # DISPLAY

In [240]:
# display(sure_bg)

In [241]:
# DISTANCE TRANSFORM
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5) # DISPLAY

In [242]:
# display(dist_transform)

In [243]:
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0) # DISPLAY

In [244]:
# display(sure_fg)

In [245]:
sure_fg = np.uint8(sure_fg)

In [246]:
unknown = cv2.subtract(sure_bg,sure_fg) # DISPLAY

In [247]:
# display(unknown)

In [248]:
# CREATING MARKERS => 3 STEPS

# 1. GETTING MARKERS

ret, markers = cv2.connectedComponents(sure_fg)

In [249]:
markers = markers + 1

In [250]:
markers[unknown == 255] = 0  # DISPLAY

In [251]:
# display(markers)

In [252]:
markers = cv2.watershed(img,markers)   # DISPLAY

In [253]:
# display(markers)

In [254]:
image, contours, hierarchy = cv2.findContours(markers.copy(),cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [255]:
for i in range(len(contours)) :
    if hierarchy[0][i][3] == -1 :
        cv2.drawContours(ct_liver,contours,i,(255,0,0),1)

In [256]:
# display(ct_liver)

In [257]:
# contours

In [258]:
mask = np.zeros(ct_liver.shape, np.uint8)
largest_areas = sorted(contours, key=cv2.contourArea)
a = cv2.drawContours(mask, [largest_areas[-3]], 0, (255,255,255,255), -1)
# removed = cv2.add(ct_liver, mask)

# cv2.imwrite("removed.png", removed)

In [259]:
cv2.imwrite(folder_path+'//out'+'//liver_mask//'+filename,mask)

True

In [260]:
final_im = mask*img2

In [261]:
cv2.imwrite(folder_path+'//out'+'//liver//'+filename,final_im)

True

In [262]:
res_img = final_im.copy()

In [263]:
# res_img = medpy.filter.smoothing.anisotropic_diffusion(final_im,niter=5, kappa=25, gamma=0.1, voxelspacing=None, option=1)

In [264]:
# display(res_img)

In [265]:
kernel = np.ones((3,3),dtype=np.uint8)
final_result = cv2.erode(final_im,kernel,iterations=1)

In [266]:
# display(final_result)

In [267]:
hsv = cv2.cvtColor(final_result,cv2.COLOR_BGR2HSV)

In [268]:
hsv[:,:,2] = cv2.equalizeHist(hsv[:,:,2])

In [269]:
res = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

In [270]:
# display(res)

In [271]:
outp = cv2.subtract(final_im,res)

In [272]:
# display(outp)

In [273]:
outp_gray = cv2.cvtColor(outp,cv2.COLOR_BGR2GRAY)

In [274]:
ret, ct_thresh = cv2.threshold(outp_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)  # DISPLAY

In [275]:
# display(ct_thresh)

In [276]:
# NOISE REMOVAL (OPTIONAL)
kernel = np.ones((3,3),np.uint8)

In [277]:
opening = cv2.morphologyEx(ct_thresh,cv2.MORPH_OPEN,kernel,iterations = 1)     # DISPLAY

In [278]:
# display(opening)

In [279]:
sure_bg = cv2.dilate(opening,kernel,iterations=3)  # DISPLAY

In [280]:
# display(sure_bg)

In [281]:
# DISTANCE TRANSFORM
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5) # DISPLAY

In [282]:
# display(dist_transform)

In [283]:
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0) # DISPLAY

In [284]:
# display(sure_fg)

In [285]:
sure_fg = np.uint8(sure_fg)

In [286]:
unknown = cv2.subtract(sure_bg,sure_fg) # DISPLAY

In [287]:
# display(unknown)

In [288]:
# CREATING MARKERS => 3 STEPS

# 1. GETTING MARKERS

ret, markers = cv2.connectedComponents(sure_fg)

In [289]:
markers = markers + 1

In [290]:
markers[unknown == 255] = 0  # DISPLAY

In [291]:
# display(markers)

In [292]:
markers = cv2.watershed(img,markers)   # DISPLAY

In [293]:
# display(markers)

In [294]:
image, contours, hierarchy = cv2.findContours(markers.copy(),cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [295]:
for i in range(len(contours)) :
    if hierarchy[0][i][3] == -1 :
        cv2.drawContours(final_im,contours,i,(255,0,0),1)

In [296]:
# display(final_im)

In [297]:
mask = np.zeros(final_im.shape, np.uint8)
largest_areas = sorted(contours, key=cv2.contourArea)
b = largest_areas[:-1]
for i in b :
    a = cv2.drawContours(mask,[i],-1,(255,255,255),-1)
# removed = cv2.add(ct_liver, mask)

# cv2.imwrite("removed.png", removed)

In [298]:
cv2.imwrite(folder_path+'//out'+'//liver_tumor_mask//'+filename,mask)

True

In [299]:
result = mask*res_img

In [300]:
cv2.imwrite(folder_path+'//out'+'//liver_tumor//'+filename,result)

True